With this nb, I played around with the grouping & packing algorithm in hqq (for axis=0), to understand how many data values are needed to process a block in hqq.

**Result:** I still don't know.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from math import ceil
cdiv = lambda x,y: ceil(x/y)

In [ ]:
m,n = 4,4
gz,pz = 2,3

ng = m*n/gz
np_= cdiv(ng, pz)

assert ng%1==0
ng = int(ng)
print(f'{ng = }, {np_ = }')

ng = 8, np_ = 3


In [ ]:
def plot(mat, title):
    x,y = mat.shape
    plt.figure(figsize=(y*0.5, x*0.5))
    plt.title(title)
    plt.imshow(mat, extent=[0, y, 0, x], vmin=0, vmax=1, aspect='equal', origin='lower')
    plt.xticks(np.arange(0, y + 1, 1))
    plt.yticks(np.arange(0, x + 1, 1))
    plt.gca().invert_yaxis()
    plt.grid(True)
    plt.show()

In [ ]:
def orig_with_selected_rows(lo,hi, m,n):
    mat = np.zeros((m,n))
    mat[lo:hi+1,:] = 1
    return mat

def orig2quant(orig, gz, axis): return orig.reshape(gz,-1) if axis==0 else orig.reshape(-1, gz)

def quant2pad(quant, ng, pz):
    gz,ng = quant.shape
    npacks = cdiv(gz,pz)
    padded = np.zeros((npacks*pz, ng))
    padded[:gz,:ng] = quant
    return padded

def pad2pack(pad, pz):
    npacks = int(pad.shape[0]/pz)
    ng = pad.shape[1]
    pack = np.zeros((npacks, ng))
    for i in range(npacks*pz): pack[i%npacks,:] += pad[i,:]
    return pack

In [ ]:
def plot_all(m, n, gz, pz, row_lo, row_hi, axis):
    assert axis in {0,1}, 'axis must be 0 or 1'
    assert row_lo<=row_hi, f'row_lo ({row_lo}) must be <= row_hi ({row_hi})'
    assert m*n%gz==0, f'group_size {gz} must divde m ({m}) or n ({n})'
    ng = int(m*n/gz)

    orig  = orig_with_selected_rows(row_lo, row_hi, m,n)
    quant = orig2quant(orig, gz, axis)
    pad   = quant2pad(quant ,ng, pz)
    pack  = pad2pack(pad, pz)
    
    for mat,title in zip([orig, quant, pad, pack], ['W', 'Quanted W', 'Padded W', 'Packed W']): plot(mat, title)

In [ ]:
from functools import partial
from ipywidgets import interact, IntSlider, Dropdown

n_slider=IntSlider(4, 1, 20, 1, description='n:') # val, min, max, step
m_slider=IntSlider(4, 1, 20, 1, description='m:')

pz_slider=IntSlider(3, 1, 10, 1, description='pack size:')
gz_slider=IntSlider(2, 1, 4, 1, description='group size:')

row_hi_slider=IntSlider(0, 0, 4, 1, description='row_end:')
row_lo_slider=IntSlider(0, 0, 0, 1, description='row_start:')

def set_max(*args, slider, ref_slider, offset=0): slider.max=ref_slider.value+offset

m_slider.observe(partial(set_max, slider=row_hi_slider, ref_slider=m_slider, offset=-1), 'value')
row_hi_slider.observe(partial(set_max, slider=row_lo_slider, ref_slider=row_hi_slider), 'value')
m_slider.observe(partial(set_max, slider=gz_slider, ref_slider=m_slider), 'value')

axis_dropdown = Dropdown(
    options=[0, 1],
    value=1,
    description='Axis:',
)

interact(
    plot_all,
    row_hi=row_hi_slider,
    row_lo=row_lo_slider,
    pz=pz_slider,
    gz=gz_slider,
    n=n_slider,
    m=m_slider,
    axis=axis_dropdown
);

interactive(children=(IntSlider(value=4, description='m:', max=20, min=1), IntSlider(value=4, description='n:'…